In [11]:
import pickle
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# Load the models
with open(r"models\random_forest_model.pkl", "rb") as f:
    rf_model = pickle.load(f)
with open(r"models\xgb_model_1.pkl", "rb") as f:
    xgb_model = pickle.load(f)
print("Models loaded!")

UnpicklingError: STACK_GLOBAL requires str